# Definitions and Imports


In [0]:

PATH_MODELS = "./Data/pointclouds/"	
PATH_NETWORK = "./trained_nets/"	
PATH_SAVED_DATA = "./Data/numpy_data/"
PATH_RESULTS = "./trained_nets/results/"


CLASS = ['aeroplane', 'bicycle', 'boat', 'bottle', 'bus', 'car', 'chair', 'diningtable', 'motorbike', 'sofa', 'train', 'tvmonitor']

EXPERIMENT_NAME = 'default'

NAME_RESULT_FILE = 'default_results'	



# if this is true, there is no embedding and the whole first stage is not run
WITHOUT_EMBEDDING = False

# if disentangled, the embedding is split into two parts:
# one for the shape label prediction and one for the pose estimation
DISENTANGLED = True


EMBEDDING_SIZE = 1024 #total embedding size
# the SHAPE_EMBEDDING_SIZE is only relevant if DISENTANGLED = False.
SHAPE_EMBEDDING_SIZE = int(EMBEDDING_SIZE / 2) # must be smaller than EMBEDDING_SIZE 


# in PASCAL3D+ there is a maximum number of 10 shapes per class category
MAX_NUM_SHAPES = 10

# number of bins on one axis
NUM_OF_BINS = 16

# alpha and beta correspond to eqn (11) in the Project Report
ALPHA = 0.5 #activation
BETA = 0.7 #inclusion radius (in radian)


# the weights of the losses correspond to eqn (15) and (16) in the Project Report
WEIGHT_DELTA_LOSS = 0.6
WEIGHT_EMBEDDING_LOSS = 0.4
WEIGHT_SHAPE_LOSS = 0.01
WEIGHT_SHAPE_LOSS_VOXNET = 0.01

# the jitter factor relates to the amount of distortion on objects 
# in the second stage, by rescaling the bounding box
JITTER_FACTOR = 0.05


# There is one dropout layer in the resnet - this is its dropout rate
DROPOUT_RATE = 0.25

# if true, this adds two dropout layers to the voxnet. 
# one before the embedding and one after
DROPOUT_VOXNET = False
DROPOUT_RATE_VOXNET = 0.5


BATCH_SIZE_VOXNET = 512
BATCH_SIZE_RESNET = 20

NUM_EPOCHS_VOXNET = 3000
NUM_EPOCHS_RESNET = 50

LR_VOXNET = 0.001	
LR_RESNET = 0.0001	

# in these epochs, the learning rate is halfed
EPOCH_CHANGE_LR_VOXNET = 3000	
EPOCH_CHANGE_LR_RESNET = 35


# if a pretrained voxnet should be loaded, this should be true, 
# and the name of the pretrained voxnet should be defined in VOXNET_LOADED_FROM
LOAD_VOXNET = False	
VOXNET_LOADED_FROM = 'nan'	

# if a pretrained resnet should be loaded, this should be true, 
# and the name of the pretrained resnet should be defined in RESNET_LOADED_FROM
LOAD_RESNET = False	
RESNET_LOADED_FROM = 'nan'	


The following code is for command line arguments, if this is run within a notebook, ignore this cell

In [0]:
import argparse	

parser = argparse.ArgumentParser(description='Training and Testing \n-> please give the following arguments:')	

parser.add_argument('--name', '-n',	
                    help='experiment name, required', required=True, metavar="EXPERIMENT_NAME")	
parser.add_argument('--weight_delta_loss', '-wdl', type=float,	
                    help='weight_delta_loss', default = WEIGHT_DELTA_LOSS, metavar="WEIGHT_DELTA_LOSS")	
parser.add_argument('--resnet_shapeloss', '-rsl', type=float,	
                    help='weight_shape_loss', default = WEIGHT_SHAPE_LOSS, metavar="WEIGHT_SHAPE_LOSS")	
parser.add_argument('--voxnet_shapeloss', '-vsl', type=float,	
                    help='weight_shape_loss_voxnet', default = WEIGHT_SHAPE_LOSS_VOXNET, metavar="WEIGHT_SHAPE_LOSS_VOXNET")	
parser.add_argument('--num_epochs_resnet', '-ner', type=int,	
                    help='num_epochs_resnet', default = NUM_EPOCHS_RESNET, metavar="NUM_EPOCHS_RESNET")	
parser.add_argument('--num_epochs_voxnet', '-nev', type=int,	
                    help='num_epochs_voxnet', default = NUM_EPOCHS_VOXNET, metavar="NUM_EPOCHS_VOXNET")	

parser.add_argument('--epoch_change_lr_resnet', '-lrr_change', type=int,	
                    help='epoch_change_lr_resnet', default = EPOCH_CHANGE_LR_RESNET, metavar="EPOCH_CHANGE_LR_RESNET")	
parser.add_argument('--epoch_change_lr_voxnet', '-lrv_change', type=int,	
                    help='epoch_change_lr_voxnet', default = EPOCH_CHANGE_LR_VOXNET, metavar="EPOCH_CHANGE_LR_VOXNET")	


parser.add_argument('--batch_size_resnet', '-bsr', type=int,	
                    help='batch_size_resnet', default = BATCH_SIZE_RESNET, metavar="BATCH_SIZE_RESNET")	
parser.add_argument('--batch_size_voxnet', '-bsv', type=int,	
                    help='batch_size_voxnet', default = BATCH_SIZE_VOXNET, metavar="BATCH_SIZE_VOXNET")	

parser.add_argument('--embedding_size', '-embsize', type=int,	
                    help='embedding_size', default = EMBEDDING_SIZE, metavar="EMBEDDING_SIZE")

parser.add_argument('--weight_emb_loss', '-wel', type=float,	
                    help='weight_embedding_loss', default = WEIGHT_EMBEDDING_LOSS, metavar="WEIGHT_EMBEDDING_LOSS")	
parser.add_argument('--without_embedding', '-wo_emb',	
                    help='without_embedding', default = WITHOUT_EMBEDDING, action='store_true')	

parser.add_argument('--disentangled', '-de',	
                    help='disentangled', default = DISENTANGLED, action='store_true')	

parser.add_argument('--alpha', '-a', type=float,	
                    help='alpha', default = ALPHA, metavar="ALPHA")	
parser.add_argument('--beta', '-b', type=float,	
                    help='beta', default = BETA, metavar="BETA")	
parser.add_argument('--lr_resnet', '-lrr', type=float,	
                    help='lr_resnet', default = LR_RESNET, metavar="LR_RESNET")	
parser.add_argument('--lr_voxnet', '-lrv', type=float,	
                    help='lr_voxnet', default = LR_VOXNET, metavar="LR_VOXNET")	
parser.add_argument('--voxnet_loaded_from', '-vload',	
                    help='if specified, the voxnet is loaded from this, and is not trained', default = VOXNET_LOADED_FROM, metavar="VOXNET_LOADED_FROM")	
parser.add_argument('--resnet_loaded_from', '-rload',	
                    help='if specified, the resnet is loaded from this, and is still trained for NUM_EPOCHS_RESNET epochs', default = RESNET_LOADED_FROM, metavar="RESNET_LOADED_FROM")	
parser.add_argument('--name_result_file', '-n_res',	
                    help='name of the file, where results are saved to as a pandas df', default = NAME_RESULT_FILE, metavar="NAME_RESULT_FILE")	

parser.add_argument('--jitter_factor', '-jf', type=float,	
                    help='jitter_factor', default = JITTER_FACTOR, metavar="JITTER_FACTOR")	

parser.add_argument('--dropout_rate', '-dpr', type=float,	
                    help='dropout_rate', default = DROPOUT_RATE, metavar="DROPOUT_RATE")	


parser.add_argument('--single_class', '-single_cls',	
                    help='if specified only this class is trained', default = 'nan', metavar="CLASS")	


parser.add_argument('--num_of_bins', '-nbins', type=int,	
                    help='num_of_bins', default = NUM_OF_BINS, metavar="NUM_OF_BINS")



args = parser.parse_args()	

EXPERIMENT_NAME = args.name	
WEIGHT_DELTA_LOSS = args.weight_delta_loss	
WEIGHT_SHAPE_LOSS = args.resnet_shapeloss	
WEIGHT_SHAPE_LOSS_VOXNET = args.voxnet_shapeloss	
NUM_EPOCHS_RESNET = args.num_epochs_resnet	
NUM_EPOCHS_VOXNET = args.num_epochs_voxnet	

EPOCH_CHANGE_LR_RESNET = args.epoch_change_lr_resnet
EPOCH_CHANGE_LR_VOXNET = args.epoch_change_lr_voxnet

BATCH_SIZE_RESNET = args.batch_size_resnet
BATCH_SIZE_VOXNET = args.batch_size_voxnet

EMBEDDING_SIZE = args.embedding_size
SHAPE_EMBEDDING_SIZE = int(EMBEDDING_SIZE / 2)

WEIGHT_EMBEDDING_LOSS = args.weight_emb_loss	
WITHOUT_EMBEDDING = args.without_embedding	

DISENTANGLED = args.disentangled

ALPHA = args.alpha	
BETA = args.beta	
LR_RESNET = args.lr_resnet	
LR_VOXNET = args.lr_voxnet	

VOXNET_LOADED_FROM = args.voxnet_loaded_from	
if VOXNET_LOADED_FROM != 'nan':	
  LOAD_VOXNET = True	
  print('Voxnet loaded from: ', VOXNET_LOADED_FROM)	

RESNET_LOADED_FROM = args.resnet_loaded_from	
if RESNET_LOADED_FROM != 'nan':	
  LOAD_RESNET = True	
  print('Resnet loaded from: ', RESNET_LOADED_FROM)	

NAME_RESULT_FILE = args.name_result_file	

JITTER_FACTOR = args.jitter_factor

DROPOUT_RATE = args.dropout_rate

if args.single_class != 'nan':
  CLASS = [args.single_class]

NUM_OF_BINS = args.num_of_bins


In [0]:
if WITHOUT_EMBEDDING:	
  NUM_EPOCHS_VOXNET = 0	

## Imports

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
print("Torch version: {}".format(torch.__version__))

Torch version: 1.3.1


In [0]:
# !grep Model: /proc/driver/nvidia/gpus/*/information | awk '{$1="";print$0}'

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


#Load the data



In [0]:
shapes_per_class = {'aeroplane':8,
                    'bicycle':6,
                    'boat':6,
                    'bottle':8,
                    'bus':6,
                    'car':10,
                    'chair':10,
                    'diningtable':6,
                    'motorbike':5,
                    'sofa':6,
                    'train':4,
                    'tvmonitor':4}

In [0]:
def load_data(classes, shapes_per_class, split='Train/'):
  valid_im_ids = np.array([])
  shape_idxs = np.array([])
  true_rots = np.zeros((0,3,3))
  bboxes = np.zeros((0,4))
  images = np.array([])
  pointclouds = []

  num_images = {}

  for cls_idx, cls in enumerate(classes):

    # pointclouds:
    pointclouds_cls = []
    for i in range(shapes_per_class[cls]):
      model = np.load(PATH_MODELS + cls + '{}.npy'.format(i), allow_pickle=True)
      pointclouds_cls.append(model)
    pointclouds.append(pointclouds_cls)
    
    # images and annotations:
    new_valid_im_ids = np.load(PATH_SAVED_DATA + split + cls + '/valid_im_ids.npy')
    num_images[cls] = new_valid_im_ids.shape[0]
    valid_im_ids = np.append(valid_im_ids, new_valid_im_ids)

    new_shape_idxs = np.load(PATH_SAVED_DATA + split + cls + '/shape_idxs.npy')	
    shape_idxs = np.append(shape_idxs, new_shape_idxs)	
    if np.max(new_shape_idxs) >= shapes_per_class[cls]:	
      print("ERROR - shape_idx", np.max(new_shape_idxs), "is too big for class:", cls)	
    
    true_rots = np.append(true_rots, np.load(PATH_SAVED_DATA + split + cls + '/true_rots.npy'), axis=0)
    bboxes = np.append(bboxes, np.load(PATH_SAVED_DATA + split + cls + '/bboxes.npy'), axis=0)

    temp_images = np.load(PATH_SAVED_DATA + split + cls + '/images.npy', allow_pickle=True)
    images = np.append(images, temp_images)    

  return pointclouds, valid_im_ids, shape_idxs, true_rots, bboxes, images, num_images
  


Load the training data:

In [0]:
pointclouds, valid_im_ids, shape_idxs, true_rots, bboxes, images, num_train_images = load_data(CLASS, shapes_per_class, split='Train/')

# Prep

## rotational bins

make the rotational bins for a 3d rotation

currently used :
- bins_6: this is a (6 x NUM_BINS) tensor, where each bin is one column, corresponding to the first two rows of a rotational matrix

helper arrays : 
- centres: rot_x and rot_z for all the bins
- centres_rot_y: camera rotation (these are 'applied' on each of the centres)


In [0]:
def create_rotation_from_xyz(rot_camera, rot_z, rot_x):
  # random rotation matrix
  rot_matrix_z = np.array([[np.cos(rot_z), -np.sin(rot_z), 0.],
                          [np.sin(rot_z), np.cos(rot_z), 0.],
                          [0., 0., 1.]])
  rot_matrix_x = np.array([[1., 0., 0.],
                          [0., np.cos(rot_x), -np.sin(rot_x)],
                          [0., np.sin(rot_x), np.cos(rot_x)]])
  rot_matrix_camera = np.array([[1., 0., 0.],
                                [0., np.cos(rot_camera), -np.sin(rot_camera)],
                                [0., np.sin(rot_camera), np.cos(rot_camera)]])

  return np.matmul(np.matmul(rot_matrix_camera, rot_matrix_z), rot_matrix_x)


BINS = NUM_OF_BINS
centres = []
for i in range(int(BINS / 2)):
  rot_z = np.pi / (BINS / 2) * (i + 0.5)
  circumpherence = 2 * np.sin(rot_z) * np.pi
  next_range = int(circumpherence / (2*np.pi / BINS))
  # print(next_range)
  for j in range(int(next_range)):
    rot_x = 2 * np.pi / (next_range) * j
    centres.append(np.array([rot_z, rot_x]))

centres_rot_y = []
for k in range(int(BINS / 2)):
  rot_y = 2 * np.pi / (BINS / 2) * (k - BINS/4 + 1)
  centres_rot_y.append(rot_y)

bins_6 = []
for [z, x] in centres:
  for cam in centres_rot_y:
    matrix = create_rotation_from_xyz(cam , z, x)
    est = matrix[:2,:].flatten()
    bins_6.append(est)
    
bins_6 = torch.transpose(torch.FloatTensor(bins_6), 0, 1)
# print(bins_6.shape)
 

## Helper functions
  - matrix_to_vec6() converts a rotation matrix to its corresponding 6-vector representation
  - vec6_to_matrix() converts a 6-vector representation to its corresponding rotation matrix
  - get_geodesic_distance() returns the geodesic distance between two rotation matrices

In [0]:
def matrix_to_vec6(matrix):
  vec6 = matrix[:2,:].flatten()
  return vec6

def vec6_to_matrix(vec6):
  first_row = vec6[:3] / torch.norm(vec6[:3])
  second_row = vec6[3:] - (torch.dot(vec6[3:], first_row) * first_row)
  second_row /= torch.norm(second_row)
  third_row = torch.cross(first_row, second_row)
  matrix = torch.cat((first_row[None,:], second_row[None,:], third_row[None,:]))
  return matrix

# not used, but here to show the concept:
# def get_geodesic_distance(R1, R2):
#   return (torch.acos((torch.trace(torch.matmul(R1, torch.transpose(R2, 0, 1))) - 1) / 2)).data.numpy()

## Functions to run on the datasets



### model manipulations

- create_random_rotation
- rotate_pointcloud
- render_volume

In [0]:
def create_random_rotation(sample):
  # random rotation matrix
  rot_z = 2*np.pi * torch.rand([1])[0]
  rot_matrix_z = torch.tensor([[torch.cos(rot_z), -torch.sin(rot_z), 0.],
                               [torch.sin(rot_z), torch.cos(rot_z), 0.],
                               [0., 0., 1.]])
  rot_x = 2*np.pi * torch.rand([1])[0]
  rot_matrix_x = torch.tensor([[1., 0., 0.],
                               [0., torch.cos(rot_x), -torch.sin(rot_x)],
                               [0., torch.sin(rot_x), torch.cos(rot_x)]])
  rot_camera = 2*np.pi * torch.rand([1])[0]
  rot_matrix_camera = torch.tensor([[1., 0., 0.],
                                    [0., torch.cos(rot_camera), -torch.sin(rot_camera)],
                                    [0., torch.sin(rot_camera), torch.cos(rot_camera)]])

  sample['orientation'] = torch.matmul(torch.matmul(rot_matrix_camera, rot_matrix_z), rot_matrix_x)
  return sample


def rotate_pointcloud(sample):
  sample['points'] = torch.matmul(sample['points'], sample['orientation'])
  return sample


def render_volume(sample):
  volume_shape = [32,32,32]
  volume_origin = torch.tensor([16,16,16])
  # in millimeter:
  voxel_size_mm = torch.tensor([1./32, 1./32, 1./32])

  # in grid raster
  points_vox = sample['points'] / voxel_size_mm + volume_origin

  # get only the points that are within our 32x32x32 grid
  points_valid = torch.BoolTensor(points_vox < 32) & torch.BoolTensor(points_vox >= 0)
  points_valid = torch.all(points_valid, axis=1)

  points_vox = (points_vox[points_valid, :]).type(torch.LongTensor)

  num_points = points_vox.size()[0]

  volume = torch.zeros(volume_shape)
  volume[points_vox[:,0], points_vox[:,1], points_vox[:,2]] = 1
  
  sample['volume'] = (volume > 0.5).type(torch.FloatTensor)

  return sample


### pose ground truths
- geodesic_distances_from_bins() calculates the distances from one orientation to each bin
- get_bin_activations() corresponds to eqn (11) in the Project report:

 $\text{for } 0 \leq i < N:
    v_i = 
    \begin{cases}
    \alpha & \text{if } GD(R_i, R_{gt}) < \beta \\
    0 & \text{otherwise}
    \end{cases}$

 where $R_{gt}$ is the ground truth rotation, $R_i$ is the rotation of the $i$th bin, $v_i$ is the activation value of the $i$th bin, $N$ is the number of bins, $\beta$ is the inclusion radius and $GD()$ is the geodesic distance.

- get_true_delta_rotations() calculates delta rotations from each bin to the orientation

In [0]:
bins_matrix = torch.zeros((bins_6.shape[1], 3, 3))
for i in range(bins_6.shape[1]):
  bins_matrix[i,:,:] = vec6_to_matrix(bins_6[:,i])

def geodesic_distances_from_bins(sample):
  # epsilon = 1e-10
  cos_distances = (torch.sum(bins_matrix.flatten(start_dim=1) * sample['orientation'].flatten(), dim=1) - 1) / 2
  distances = torch.acos(torch.min(torch.max(cos_distances, -1 * torch.ones(cos_distances.shape)), torch.ones(cos_distances.shape)))
  sample['geodesic_distances'] = distances
  return sample

def get_bin_activations(sample):
  sample = geodesic_distances_from_bins(sample)
  sample['rot_bin_activations'] = ALPHA * (sample['geodesic_distances'] < BETA).float()
  if(torch.sum(sample['rot_bin_activations']) == 0):
    print("ERROR --> all bin activations are zero: increase the inclusion radius beta and check whether your rotation matrices are valid.")
  return sample


def get_true_delta_rotations(sample):
  rot_d = torch.transpose(torch.matmul(bins_matrix, sample['orientation']).flatten(start_dim=1), 0, 1)
  sample['delta_rots'] = rot_d
  return sample

## Evaluation functions
- geodesic_distances_batch_from_rots() returns a tensor of geodesic distances and an accurate_count for a whole batch
- med_error_batch_from_rots() returns the median of all geodesic distances between prediction and ground true rotations of a whole batch and the corresponding accuracy



In [0]:
def geodesic_distances_batch_from_rots(batch, results):

  best_bin_idxs = torch.argmax(results['rot_bins'].to("cpu"), dim=1)

  rows_x = results['rot_deltas_x'][np.arange(len(results['rot_deltas_x'])), :, best_bin_idxs].data
  rows_y = results['rot_deltas_y'][np.arange(len(results['rot_deltas_y'])), :, best_bin_idxs].data
  rows_z = results['rot_deltas_z'][np.arange(len(results['rot_deltas_z'])), :, best_bin_idxs].data

  predicted_delta_rot = torch.cat((rows_x[:,None,:], rows_y[:,None,:], rows_z[:,None,:]), dim=1).to("cpu").data

  predicted_rot = torch.matmul(torch.transpose(bins_matrix[best_bin_idxs,:,:], 1, 2), predicted_delta_rot)


  cos_distances = (torch.sum(predicted_rot.flatten(start_dim=1) * batch['orientation'].flatten(start_dim=1).to("cpu"), dim=1) - 1) / 2
  geodesic_distances = torch.acos(torch.min(torch.max(cos_distances, -1 * torch.ones(cos_distances.shape)), torch.ones(cos_distances.shape)))
  
  accurate_count = torch.sum(geodesic_distances < 0.52)

  return geodesic_distances, accurate_count



def med_error_batch_from_rots(batch, results):
  geodesic_distances, accurate_count = geodesic_distances_batch_from_rots(batch, results)
  return torch.median(geodesic_distances).data, accurate_count.float() / batch['orientation'].shape[0]

# STAGE 1: voxel grid to embedding to shape label and pose

## create Dataset and apply functions
dataset is a dict within a pytorch dataset

after applying functions it has the following elements:
- **class_index** and **shape_index**: randomly selected
- **orientation**: random 3x3 rotation
- (**points**: pointcloud of shape given by **class_idx** and **shape_idx** in the rotation given by **orientation**.
 -> Note that the points are deleted again after the rest of the data has been generated.)

- **volume**: 32x32x32 volume rendered from rotated pointcloud
- **rot_bin_activations**: true activation of rotational bins (given by eqn (11))
- **geodesic_distances**: list of geodesic distances between **orientation** and all bins
- **delta_rots**: list of delta rotations from each bin to the **orientation**


In [0]:
class DatasetVoxNet(torch.utils.data.Dataset):
  def __init__(self):
    self.pointclouds = pointclouds
  
  def __len__(self):
    return BATCH_SIZE_VOXNET

  def __getitem__(self, index):
    class_index = torch.randint(0,len(CLASS),[1])[0]
    shape_index = torch.randint(0, shapes_per_class[CLASS[class_index]], [1])[0]

    sample = {}
    sample['class_index'] = class_index
    sample['shape_index'] = shape_index
    sample['points'] = torch.FloatTensor(self.pointclouds[class_index][shape_index]) * 1.3

    sample = create_random_rotation(sample) # adds 'orientation'
    sample = rotate_pointcloud(sample) # modifies 'points'
    sample = render_volume(sample) # adds 'volume'

    sample = get_bin_activations(sample) # adds 'rot_bin_activations' and 'geodesic_distances'
    sample = get_true_delta_rotations(sample) # adds 'delta_rots'

    # delete points because the 7 different models have different numbers of points and torch cant deal with that
    del sample['points']
    return sample

In [0]:
dataloader = torch.utils.data.DataLoader(DatasetVoxNet(), batch_size=BATCH_SIZE_VOXNET)

## Build the network architecture (VoxNet)
https://www.ri.cmu.edu/pub_files/2015/9/voxnet_maturana_scherer_iros15.pdf

In [0]:
class VoxNet(nn.Module):

  def __init__(self):
    super(VoxNet, self).__init__()
    self.num_of_bins = bins_6.shape[1]

    factor = 2

    self.conv1 = nn.Conv3d(1, 32*factor, kernel_size=(5,5,5), stride=2, padding=2) #padding='VALID')
   
    self.conv2 = nn.Conv3d(32*factor, 32*factor, kernel_size=(3,3,3), padding=1)#, padding='VALID')

    self.conv3 = nn.Conv3d(32*factor, 32*factor, kernel_size=(3,3,3), padding=1)#, padding='VALID')	

    self.pool = nn.AvgPool3d(kernel_size=(2,2,2), stride=2)

    self.linear1 = nn.Linear((32*factor)*8*8*8, EMBEDDING_SIZE)

    if DROPOUT_VOXNET:
      self.linear1 = nn.Sequential(nn.Dropout(p=DROPOUT_RATE_VOXNET), nn.Linear((32*factor)*8*8*8, EMBEDDING_SIZE))

    self.sigmoid = nn.Sigmoid()

    if DISENTANGLED:
      self.linear_shape = nn.Linear(SHAPE_EMBEDDING_SIZE, MAX_NUM_SHAPES*len(CLASS))
      self.linear_rot_bin = nn.Linear(EMBEDDING_SIZE - SHAPE_EMBEDDING_SIZE, self.num_of_bins)
      if DROPOUT_VOXNET:
        self.linear_rot_bin = nn.Sequential(nn.Dropout(p=DROPOUT_RATE_VOXNET), nn.Linear(EMBEDDING_SIZE - SHAPE_EMBEDDING_SIZE, self.num_of_bins))

      self.linear_rot_delta_x = nn.Linear(EMBEDDING_SIZE - SHAPE_EMBEDDING_SIZE, self.num_of_bins * 3)
      self.linear_rot_delta_y = nn.Linear(EMBEDDING_SIZE - SHAPE_EMBEDDING_SIZE, self.num_of_bins * 3)

    else:
      self.linear_shape = nn.Linear(EMBEDDING_SIZE, MAX_NUM_SHAPES*len(CLASS))
      self.linear_rot_bin = nn.Linear(EMBEDDING_SIZE, self.num_of_bins)
      if DROPOUT_VOXNET:
        self.linear_rot_bin = nn.Sequential(nn.Dropout(p=DROPOUT_RATE_VOXNET), nn.Linear(EMBEDDING_SIZE, self.num_of_bins))

      self.linear_rot_delta_x = nn.Linear(EMBEDDING_SIZE, self.num_of_bins * 3)
      self.linear_rot_delta_y = nn.Linear(EMBEDDING_SIZE, self.num_of_bins * 3)

    self.flatten = nn.Flatten()

    
  def __call__(self, batch):
    embedding = self.embedding_from_batch(batch)
    results = self.results_from_embedding(embedding)
    results['embedding'] = embedding
    return results

  

  def embedding_from_batch(self, batch):
    act = batch['volume'][:,None,:,:,:]

    act = self.conv1(act)
    act = F.relu(act)
    act = self.conv2(act)
    act = F.relu(act)	
    act = self.conv3(act)
    act = F.relu(act)
    act = self.pool(act)

    act = self.flatten(act)
    embedding = self.linear1(act)

    return embedding


  def results_from_embedding(self, embedding):
     # SHAPE
    if DISENTANGLED:
      class_shape = self.linear_shape(embedding[:,:SHAPE_EMBEDDING_SIZE])
    else:
      class_shape = self.linear_shape(embedding)

    # ROTATION
    if DISENTANGLED:
      rot_bins = self.sigmoid(self.linear_rot_bin(embedding[:,SHAPE_EMBEDDING_SIZE:]))
      unnormalized_rot_deltas_x = self.linear_rot_delta_x(embedding[:,SHAPE_EMBEDDING_SIZE:]).reshape((embedding.shape[0], 3, self.num_of_bins))
      unnormalized_rot_deltas_y = self.linear_rot_delta_y(embedding[:,SHAPE_EMBEDDING_SIZE:]).reshape((embedding.shape[0], 3, self.num_of_bins))
    else:
      rot_bins = self.sigmoid(self.linear_rot_bin(embedding))
      unnormalized_rot_deltas_x = self.linear_rot_delta_x(embedding).reshape((embedding.shape[0], 3, self.num_of_bins))
      unnormalized_rot_deltas_y = self.linear_rot_delta_y(embedding).reshape((embedding.shape[0], 3, self.num_of_bins))


    # this corresponds to getting a rotation matrix from a 6-vector:
    lengths_x = torch.norm(unnormalized_rot_deltas_x, dim=1, keepdim=True)
    epsilon = 10**(-10)
    unit_vectors_x = unnormalized_rot_deltas_x / (lengths_x + epsilon)
    projections = torch.sum((unit_vectors_x * unnormalized_rot_deltas_y), dim=1, keepdim=True)
    projected_rot_deltas_y = unnormalized_rot_deltas_y - projections * unit_vectors_x
    lengths_y = torch.norm(projected_rot_deltas_y, dim=1, keepdim=True)
    unit_vectors_y = projected_rot_deltas_y / (lengths_y + epsilon)

    unit_vectors_z = torch.cross(unit_vectors_x, unit_vectors_y, dim=1)

    return {'class_shape':class_shape,
            'rot_bins':rot_bins, 
            'rot_deltas_x':unit_vectors_x, 'rot_deltas_y':unit_vectors_y, 'rot_deltas_z':unit_vectors_z, 
            'lengths_x':lengths_x, 'lengths_y':lengths_y,
            'projections':projections}

## train network

In [0]:
voxNet = VoxNet()
voxNet = voxNet.to(device)

In [0]:
if LOAD_VOXNET:	
  voxNet.load_state_dict(torch.load(PATH_NETWORK + VOXNET_LOADED_FROM + '_voxNet_best_med_error.pt'))	

criterion is a binary cross entropy loss for the bins and a regression loss for the deltas

In [0]:
def criterion_voxNet(results, sample):
  # loss for bin classification
  bin_loss = torch.mean(nn.BCELoss()(results['rot_bins'], sample['rot_bin_activations']))

  predicted_deltas = torch.cat((results['rot_deltas_x'], results['rot_deltas_y'], results['rot_deltas_z']), dim=1)

  # epsilon is added here, so that we have only values in the arccos that are between -1 and 1 (otherwise we get nans..)
  # and if we just used min(max(..)) then we might be cutting off important gradients
  epsilon = 0.01

  cos_distances = (torch.sum(predicted_deltas * sample['delta_rots'], dim=1) - 1) / 2

  # loss for delta rotations
  delta_loss = torch.mean(torch.sum(
      sample['rot_bin_activations'] * 
      (torch.acos((1-epsilon) * cos_distances)), dim=1) / 
      torch.sum(sample['rot_bin_activations'], dim=1))

  # loss for lengths
  length_loss_x = torch.mean((results['lengths_x'] - 1.)**2)
  length_loss_y = torch.mean((results['lengths_y'] - 1.)**2)

  # projection loss
  projection_loss = torch.mean(results['projections']**2)
      
  return {'loss': bin_loss + WEIGHT_DELTA_LOSS * delta_loss + 0.01 * (length_loss_x + length_loss_y + projection_loss),
          'bin_loss': bin_loss,
          'delta_loss': delta_loss,
          'length_loss_x': length_loss_x,
          'length_loss_y': length_loss_y,
          'projection_loss': projection_loss}

In [0]:
# some things are saved for debugging and testing
# and keeping track of the best network

losses_voxNet = []
med_errors_voxNet = []
best_med_error_voxNet = 100
corresponding_accuracy_voxNet = 0
corresponding_correct_shapes_voxNet = 0
corresponding_correct_classes_voxNet = 0

class_shape_losses = []
bin_losses = []
delta_losses = []
length_losses_x = []
length_losses_y = []
projection_losses = []

In [0]:
criterion_classes_shapes = nn.CrossEntropyLoss()

In [24]:
opt = torch.optim.Adam(voxNet.parameters(), lr=LR_VOXNET)


def episode(batch):
  opt.zero_grad()
  # run network on batch
  results = voxNet(batch)

  # get predictions for classes and shape labels and count correct ones
  predictions_classes_shapes = (torch.argmax(results['class_shape'].to("cpu"), axis=1)).type(torch.IntTensor)
  predictions_classes = (predictions_classes_shapes / MAX_NUM_SHAPES).type(torch.IntTensor)
  correct_classes = torch.sum(torch.eq(predictions_classes, batch['class_index'].to("cpu")))
  predictions_shapes = (predictions_classes_shapes - predictions_classes * MAX_NUM_SHAPES).type(torch.IntTensor)
  correct_shapes = torch.sum(torch.eq(predictions_shapes, batch['shape_index'].to("cpu")))
  
  # get med error and accuracy between predicted pose and true pose
  med_error, accuracy = med_error_batch_from_rots(batch, results)

  # sum up losses
  all_losses = criterion_voxNet(results, batch)
  all_losses['class_shape'] = criterion_classes_shapes(results['class_shape'], batch['class_index'] * MAX_NUM_SHAPES + batch['shape_index'])
  loss = WEIGHT_SHAPE_LOSS_VOXNET * all_losses['class_shape'] + all_losses['loss']

  loss.backward()
  opt.step()

  results['loss'] = loss
  results['med_error'] = med_error
  results['accuracy'] = accuracy
  results['all_losses'] = all_losses
  results['correct_shapes'] = correct_shapes
  results['correct_classes'] = correct_classes
  return results



voxNet.train()
torch.set_grad_enabled(True)

for epoch in range(NUM_EPOCHS_VOXNET):

  # change learning rate:
  if epoch == EPOCH_CHANGE_LR_VOXNET:
    for param_group in opt.param_groups:
      param_group['lr'] /= 2


  batch = next(iter(dataloader))
  batch = {k:v.to(device) for (k,v) in batch.items()}
  results = episode(batch)

  # print results of this batch
  print("{} -- loss: {}, med_error: {}, accuracy: {}".format(epoch, results['loss'].data, results['med_error'], results['accuracy']))

  # if this was the best batch so far, save network state and some other things
  if results['med_error'] < best_med_error_voxNet:
    best_med_error_voxNet = results['med_error']
    corresponding_accuracy_voxNet = results['accuracy']
    corresponding_correct_shapes_voxNet = float(results['correct_shapes']) / BATCH_SIZE_VOXNET
    corresponding_correct_classes_voxNet = float(results['correct_classes']) / BATCH_SIZE_VOXNET
    torch.save(voxNet.state_dict(), PATH_NETWORK + EXPERIMENT_NAME + '_voxNet_best_med_error.pt')
  print("bin_loss: {}, delta_loss: {}".format(results['all_losses']['bin_loss'].data, results['all_losses']['delta_loss'].data))
  print("shape_loss = {}, correct_classes = {}, correct_shapes = {} out of {}\n".format(results['all_losses']['class_shape'].data, results['correct_classes'], results['correct_shapes'], BATCH_SIZE_VOXNET))

  bin_losses.append(results['all_losses']['bin_loss'].data)
  delta_losses.append(results['all_losses']['delta_loss'].data)
  length_losses_x.append(results['all_losses']['length_loss_x'].data)
  length_losses_y.append(results['all_losses']['length_loss_y'].data)
  projection_losses.append(results['all_losses']['projection_loss'].data)
  class_shape_losses.append(results['all_losses']['class_shape'].data)
  
  losses_voxNet.append(results['loss'].data)
  med_errors_voxNet.append(results['med_error'])




Streaming output truncated to the last 5000 lines.
bin_loss: 0.035088419914245605, delta_loss: 0.19506192207336426
shape_loss = 0.012665610760450363, correct_classes = 512, correct_shapes = 512 out of 512

1751 -- loss: 0.15705345571041107, med_error: 0.40177443623542786, accuracy: 0.5078125
bin_loss: 0.03502148389816284, delta_loss: 0.19873738288879395
shape_loss = 0.016037821769714355, correct_classes = 512, correct_shapes = 511 out of 512

1752 -- loss: 0.1550232321023941, med_error: 0.1483210325241089, accuracy: 0.53125
bin_loss: 0.035250384360551834, delta_loss: 0.195061594247818
shape_loss = 0.018257249146699905, correct_classes = 511, correct_shapes = 511 out of 512

1753 -- loss: 0.1587052047252655, med_error: 1.53767991065979, accuracy: 0.443359375
bin_loss: 0.03591185808181763, delta_loss: 0.2002171277999878
shape_loss = 0.012496672570705414, correct_classes = 512, correct_shapes = 512 out of 512

1754 -- loss: 0.1566522866487503, med_error: 0.13476049900054932, accuracy: 0.5

## display results

In [0]:
# plt.plot(losses_voxNet[100:], label='loss')
# plt.show()
# plt.plot(class_shape_losses[100:])
# plt.show()
# plt.plot(bin_losses[100:])
# plt.show()

# plt.plot(delta_losses[100:])
# plt.show()
# plt.plot(length_losses_x[100:])
# plt.show()
# plt.plot(length_losses_y[100:])
# plt.show()
# plt.plot(projection_losses[100:])

In [26]:
# fig, ax = plt.subplots(1,2)
# ax[0].plot(losses_voxNet, label='loss')
# ax[1].plot(med_errors_voxNet, label='med_error')
# ax[0].legend()
# ax[1].legend()
# ax[0].set_ylim((2.55,2.7))
# ax[1].set_ylim((4,6))
print(best_med_error_voxNet)
print(corresponding_accuracy_voxNet)

tensor(0.0674)
tensor(0.7031)


## save results for the voxnet

In [0]:
if NUM_EPOCHS_VOXNET > 0:	
  text = 'best_med_error: {}\ncorresponding_accuracy: {}\ncorresponding correct shapes: {}\ncorrsponding correct classes: {}\n'.format(best_med_error_voxNet, corresponding_accuracy_voxNet, corresponding_correct_shapes_voxNet, corresponding_correct_classes_voxNet)	
  with open(PATH_RESULTS + EXPERIMENT_NAME + '_voxNet_best', "w") as text_file:	
      text_file.write(text)
  voxNet.load_state_dict(torch.load(PATH_NETWORK + EXPERIMENT_NAME + '_voxNet_best_med_error.pt'))

  np.save(PATH_RESULTS + EXPERIMENT_NAME + '_voxNet_med_error', med_errors_voxNet)
  np.save(PATH_RESULTS + EXPERIMENT_NAME + '_voxNet_loss', losses_voxNet)


# STAGE 2: ResNet from Image to Embedding

## Helper functions (get_embedding, and transform_image)

In [0]:
def get_embedding(sample):
  temp = {}
  try:
    temp['points'] = torch.FloatTensor(pointclouds[sample['class_index']][sample['shape_index']])
  except:
    print("something went wrong with the loading of pointclouds.. \nThe class idx, shape idx it couldnt do are:")
    print(sample['class_index'], sample['shape_index'])
  temp['orientation'] = sample['orientation']
  temp = rotate_pointcloud(temp)
  temp = render_volume(temp)
  # make it into a 'batch':
  temp['volume'] = temp['volume'].unsqueeze(0)
  temp = {k:v.to(device) for (k,v) in temp.items()}
  sample['embedding'] = voxNet.embedding_from_batch(temp)[0]
  return sample


In [0]:
my_transform = transforms.Compose([
  transforms.Resize((224,224)), 
  transforms.ColorJitter(0.1,0.1,0.1,0.1),
  transforms.ToTensor(), 
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

def transform_image(sample):

  bbox_dim = max(sample['bbox'][2] - sample['bbox'][0], sample['bbox'][3] - sample['bbox'][1])	
  midpoint = [1/2*(sample['bbox'][0] + sample['bbox'][2]), 1/2*(sample['bbox'][1] + sample['bbox'][3])]

  # jittering of objects dimension:
  x_jitter = (np.random.rand() - 0.5) * JITTER_FACTOR 
  x_dim = (x_jitter + 1) * bbox_dim

  y_jitter = (np.random.rand() - 0.5) * JITTER_FACTOR 
  y_dim = (y_jitter + 1) * bbox_dim

  temp_im = transforms.ToPILImage()(sample['image_original'])
  temp_im = transforms.functional.crop(temp_im, midpoint[1] - x_dim / 2, midpoint[0] - y_dim / 2, x_dim, y_dim)
 
  sample['image'] = my_transform(temp_im).to(device)
  
  return sample


my_transform_test = transforms.Compose([
  transforms.Resize((224,224)), 
  transforms.ToTensor(), 
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

def transform_image_test(sample):

  bbox_dim = max(sample['bbox'][2] - sample['bbox'][0], sample['bbox'][3] - sample['bbox'][1])	
  midpoint = [1/2*(sample['bbox'][0] + sample['bbox'][2]), 1/2*(sample['bbox'][1] + sample['bbox'][3])]	

  temp_im = transforms.ToPILImage()(sample['image_original'])	
  temp_im = transforms.functional.crop(temp_im, midpoint[1] - bbox_dim / 2, midpoint[0] - bbox_dim / 2, bbox_dim, bbox_dim)
  
  sample['image'] = my_transform_test(temp_im).to(device)
  
  return sample


## create datasets and apply functions

datasets are dicts within pytorch datasets

after applying functions they have the following elements:
- image
- **im_id**: corresponding file id
- **class_index** and **shape_index**: corresponding shape and class info
- **bbox**: corresponding 2D bounding box
- **orientation**: corresponding orientation
- **rot_bin_activations**: true activation of rotational bins (given by eqn (11))
- **geodesic_distances**: list of geodesic distances between **orientation** and all bins
- **delta_rots**: list of delta rotations from each bin to the **orientation**
- **embedding**: obtained from the voxnet

In [0]:
VAL_SIZE = BATCH_SIZE_RESNET

class ImageDatasetTrain(torch.utils.data.Dataset):
  def __init__(self):
    self.train_im_ids = np.zeros((0,))
    self.train_true_rots = np.zeros((0,3,3))
    self.train_shape_idxs = np.zeros((0,))
    self.train_bboxes = np.zeros((0,4))
    self.images = np.zeros((0,))

    length_classes = []
    start_index = 0
    for cls in CLASS:
      length_classes.append(num_train_images[cls] - VAL_SIZE)
      end_index = start_index + num_train_images[cls]
      self.train_im_ids = np.append(self.train_im_ids, valid_im_ids[start_index:end_index-VAL_SIZE], axis=0)
      self.train_true_rots = np.append(self.train_true_rots, true_rots[start_index:end_index-VAL_SIZE], axis=0)
      self.train_shape_idxs = np.append(self.train_shape_idxs, shape_idxs[start_index:end_index-VAL_SIZE], axis=0)
      self.train_bboxes = np.append(self.train_bboxes, bboxes[start_index:end_index-VAL_SIZE], axis=0)
      self.images = np.append(self.images, images[start_index:end_index-VAL_SIZE], axis=0)
    
      start_index = end_index
      
    self.train_class_idxs = np.repeat(range(len(CLASS)), length_classes)
    self.train_shape_idxs = self.train_shape_idxs.astype(int)

  
  def __len__(self):
    return len(self.train_im_ids)

  def __getitem__(self, index):
    sample = {}
    sample['im_id'] = self.train_im_ids[index]
    sample['rot'] = self.train_true_rots[index]
    sample['shape_index'] = self.train_shape_idxs[index]
    sample['class_index'] = self.train_class_idxs[index]
    sample['bbox'] = self.train_bboxes[index]

    sample['image_original'] = self.images[index]
    sample = transform_image(sample) # adds 'image'

    del sample['image_original']

    sample['orientation'] = torch.FloatTensor(sample['rot'])
    del sample['rot']
 
    sample = get_bin_activations(sample)# adds 'rot_bin_activations' and 'geodesic_distances'
    sample = get_true_delta_rotations(sample) # adds 'delta_rots'

    sample = get_embedding(sample) # adds 'embedding'
    return sample

class ImageDatasetValidate(torch.utils.data.Dataset):
  def __init__(self):
    self.validate_im_ids = np.zeros((0,))
    self.validate_true_rots = np.zeros((0,3,3))
    self.validate_shape_idxs = np.zeros((0,))
    self.validate_bboxes = np.zeros((0,4))
    self.images = np.zeros((0,))

    length_classes = []
    start_index = 0
    for cls in CLASS:
      length_classes.append(VAL_SIZE)
      end_index = start_index + num_train_images[cls]
      self.validate_im_ids = np.append(self.validate_im_ids, valid_im_ids[end_index-VAL_SIZE:end_index], axis=0)
      self.validate_true_rots = np.append(self.validate_true_rots, true_rots[end_index-VAL_SIZE:end_index], axis=0)
      self.validate_shape_idxs = np.append(self.validate_shape_idxs, shape_idxs[end_index-VAL_SIZE:end_index], axis=0)
      self.validate_bboxes = np.append(self.validate_bboxes, bboxes[end_index-VAL_SIZE:end_index], axis=0)
      self.images = np.append(self.images, images[end_index-VAL_SIZE:end_index], axis=0)
    
      start_index = end_index
      
    self.validate_class_idxs = np.repeat(range(len(CLASS)), length_classes)
    self.validate_shape_idxs = self.validate_shape_idxs.astype(int)

  
  def __len__(self):
    return len(self.validate_im_ids)

  def __getitem__(self, index):
    sample = {}
    sample['im_id'] = self.validate_im_ids[index]
    sample['rot'] = self.validate_true_rots[index]
    sample['shape_index'] = self.validate_shape_idxs[index]
    sample['class_index'] = self.validate_class_idxs[index]
    sample['bbox'] = self.validate_bboxes[index]
 
    sample['image_original'] = self.images[index]
    sample = transform_image_test(sample) # adds 'image'

    del sample['image_original']
 
    sample['orientation'] = torch.FloatTensor(sample['rot'])
    del sample['rot']

    sample = get_bin_activations(sample)# adds 'rot_bin_activations' and 'geodesic_distances'
    sample = get_true_delta_rotations(sample) # adds 'delta_rots'

    sample = get_embedding(sample) # adds 'embedding'
    return sample

In [0]:
image_dataloader_train = torch.utils.data.DataLoader(ImageDatasetTrain(), batch_size=BATCH_SIZE_RESNET, shuffle=True)

image_dataloader_validate = torch.utils.data.DataLoader(ImageDatasetValidate(), batch_size=BATCH_SIZE_RESNET)


## Network
now load a pretrained resnet without its last layer

https://arxiv.org/pdf/1605.07146.pdf

In [0]:
resNet = torchvision.models.wide_resnet50_2(pretrained=True)

In [0]:
resNet.fc = nn.Sequential(nn.Dropout(p=DROPOUT_RATE), nn.Linear(2048, EMBEDDING_SIZE))
if LOAD_RESNET:	
  resNet.load_state_dict(torch.load(PATH_NETWORK + RESNET_LOADED_FROM + '_resNet_best_med_error.pt'))	
  torch.save(resNet.state_dict(), PATH_NETWORK + EXPERIMENT_NAME + '_resNet_best_med_error.pt')	

In [0]:
# Send the model to GPU
resNet = resNet.to(device)

## training

In [0]:
losses_val_resNet = []
losses_train_resNet = []
med_errors_val_resNet = []
best_med_error_resNet = 100
corresponding_accuracy_resNet = 0
corresponding_correct_shapes_resNet = 0
corresponding_correct_classes_resNet = 0

bin_losses = []	
delta_losses = []	
length_losses_x = []	
length_losses_y = []	
projection_losses = []	
class_shape_losses = []	
embedding_losses = []

In [36]:
voxNet.eval()

VoxNet(
  (conv1): Conv3d(1, 64, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(2, 2, 2))
  (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool): AvgPool3d(kernel_size=(2, 2, 2), stride=2, padding=0)
  (linear1): Linear(in_features=32768, out_features=1024, bias=True)
  (sigmoid): Sigmoid()
  (linear_shape): Linear(in_features=512, out_features=120, bias=True)
  (linear_rot_bin): Linear(in_features=512, out_features=624, bias=True)
  (linear_rot_delta_x): Linear(in_features=512, out_features=1872, bias=True)
  (linear_rot_delta_y): Linear(in_features=512, out_features=1872, bias=True)
  (flatten): Flatten()
)

In [0]:
if not WITHOUT_EMBEDDING:	
  params = resNet.parameters()	
else:	
  params = list(resNet.parameters()) + list(voxNet.parameters())	

opt = torch.optim.Adam(params, lr=LR_RESNET)	
criterion_embedding = nn.L1Loss()

In [0]:
def episode(batch):
  resNet.train()
  torch.set_grad_enabled(True)
  opt.zero_grad()

  # run resnet on batch
  logits = resNet(batch['image'])

  # calculate embedding loss from resnet output
  loss_embedding = criterion_embedding(logits, batch['embedding']) 

  # get bin results by running the second part of the voxnet on the output of the resnet
  bin_result = voxNet.results_from_embedding(logits)

  # losses...
  other_losses = criterion_voxNet(bin_result, batch)
  other_losses['class_shape'] = criterion_classes_shapes(bin_result['class_shape'], batch['class_index'] * MAX_NUM_SHAPES + batch['shape_index'])

  loss = WEIGHT_EMBEDDING_LOSS * loss_embedding + other_losses['loss'] + WEIGHT_SHAPE_LOSS * other_losses['class_shape']

  loss.backward()
  opt.step()

  results = {}
  results['loss'] = loss
  results['all_losses'] = other_losses	
  results['loss_embedding'] = loss_embedding
  return results


count = 0

for i in range(NUM_EPOCHS_RESNET):

  if i == EPOCH_CHANGE_LR_RESNET:
    for param_group in opt.param_groups:
      param_group['lr'] /= 2

  print(i)
  for batch in image_dataloader_train:

    batch['embedding'] = batch['embedding'].to(device)
    batch['rot_bin_activations'] = batch['rot_bin_activations'].to(device)
    batch['delta_rots'] = batch['delta_rots'].to(device)
    batch['shape_index'] = batch['shape_index'].type(torch.LongTensor).to(device)
    batch['class_index'] = batch['class_index'].type(torch.LongTensor).to(device)

    results = episode(batch)

    # every 50 batches, the validation set is evaluated
    if count % 50 == 0:

      correct_classes = 0
      correct_shapes = 0
      geodesic_distances = []
      accurate_count = 0
      sum_loss_val = 0

      num_batches_val = 0
      for batch_val in image_dataloader_validate:
        num_batches_val += 1

        batch_val['embedding'] = batch_val['embedding'].to(device)
        batch_val['rot_bin_activations'] = batch_val['rot_bin_activations'].to(device)
        batch_val['delta_rots'] = batch_val['delta_rots'].to(device)
        batch_val['shape_index'] = batch_val['shape_index'].type(torch.LongTensor).to(device)
        batch_val['class_index'] = batch_val['class_index'].type(torch.LongTensor).to(device)

        resNet.eval()
        torch.set_grad_enabled(False)
        logits_val = resNet(batch_val['image'])

        # losses
        loss_embedding_val = criterion_embedding(logits_val, batch_val['embedding']) 
        bin_result_val = voxNet.results_from_embedding(logits_val)
        other_losses_val = criterion_voxNet(bin_result_val, batch_val)
        other_losses_val['class_shape'] = criterion_classes_shapes(bin_result_val['class_shape'], batch_val['class_index'] * MAX_NUM_SHAPES + batch_val['shape_index'])

        sum_loss_val += WEIGHT_EMBEDDING_LOSS * loss_embedding_val + other_losses_val['loss'] + WEIGHT_SHAPE_LOSS * other_losses_val['class_shape']

        # pose errors and accuracy
        gds, acc_count = geodesic_distances_batch_from_rots(batch_val, bin_result_val)
        geodesic_distances.extend(gds)
        accurate_count += acc_count

        # class and shape prediction
        predictions_classes_shapes = (torch.argmax(bin_result_val['class_shape'].to("cpu"), axis=1)).type(torch.IntTensor)
        predictions_classes = (predictions_classes_shapes / MAX_NUM_SHAPES).type(torch.IntTensor)
        correct_classes += torch.sum(torch.eq(predictions_classes, batch_val['class_index'].to("cpu")))
        predictions_shapes = (predictions_classes_shapes - predictions_classes * MAX_NUM_SHAPES).type(torch.IntTensor)
        correct_shapes += torch.sum(torch.eq(predictions_shapes, batch_val['shape_index'].to("cpu")))

      med_error_val = torch.median(torch.FloatTensor(geodesic_distances))
      accuracy_val = float(accurate_count) / len(image_dataloader_validate.dataset)
      shape_accuracy_val = float(correct_shapes) / len(image_dataloader_validate.dataset)
      class_accuracy_val = float(correct_classes) / len(image_dataloader_validate.dataset)


      print("{} -- training loss: {}\nvalidation loss: {}\ngeodesic distance: {}\naccuracy: {}\ncorrect classes: {}\ncorrect shapes: {}\n".format(i, results['loss'].data, sum_loss_val.data, med_error_val, accuracy_val, class_accuracy_val, shape_accuracy_val))
      losses_val_resNet.append(sum_loss_val.data / num_batches_val)
      med_errors_val_resNet.append(med_error_val)

      if med_error_val < best_med_error_resNet:
        best_med_error_resNet = med_error_val
        corresponding_accuracy_resNet = accuracy_val
        corresponding_correct_shapes_resNet = shape_accuracy_val
        corresponding_correct_classes_resNet = class_accuracy_val
        torch.save(resNet.state_dict(), PATH_NETWORK + EXPERIMENT_NAME + '_resNet_best_med_error.pt')
        np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_med_error_val_process', med_errors_val_resNet)
        np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_loss_val_process', losses_val_resNet)

        if WITHOUT_EMBEDDING:	
          torch.save(voxNet.state_dict(), PATH_NETWORK + EXPERIMENT_NAME + '_voxNet_best_med_error.pt')

    else:
      print("{} -- training loss: {}\n".format(i, results['loss'].data))


    bin_losses.append(results['all_losses']['bin_loss'].data)	
    delta_losses.append(results['all_losses']['delta_loss'].data)	
    length_losses_x.append(results['all_losses']['length_loss_x'].data)	
    length_losses_y.append(results['all_losses']['length_loss_y'].data)	
    projection_losses.append(results['all_losses']['projection_loss'].data)	
    class_shape_losses.append(results['all_losses']['class_shape'].data)	
    embedding_losses.append(results['loss_embedding'].data)

    losses_train_resNet.append(results['loss'].data)
    count += 1

Streaming output truncated to the last 5000 lines.

21 -- training loss: 0.20852570235729218

21 -- training loss: 0.2218177765607834

21 -- training loss: 0.19827471673488617

21 -- training loss: 0.2245374470949173

21 -- training loss: 0.2104145735502243
validation loss: 3.630127191543579
geodesic distance: 0.15318599343299866
accuracy: 0.8708333333333333
correct classes: 0.9125
correct shapes: 0.6166666666666667

21 -- training loss: 0.2127639800310135

21 -- training loss: 0.21751274168491364

21 -- training loss: 0.2051393985748291

21 -- training loss: 0.226973757147789

21 -- training loss: 0.21590262651443481

21 -- training loss: 0.22386585175991058

21 -- training loss: 0.22488786280155182

21 -- training loss: 0.21251818537712097

21 -- training loss: 0.21204355359077454

21 -- training loss: 0.2208474725484848

21 -- training loss: 0.2063179314136505

21 -- training loss: 0.20947206020355225

21 -- training loss: 0.2252701073884964

21 -- training loss: 0.2393369823694229


## display results

In [0]:
# fig, ax = plt.subplots(1, 2)
# ax[0].plot(losses_train_resNet, label='train')
# ax[0].plot(losses_val_resNet, label='val')
# ax[1].plot(med_errors_val_resNet, label='med error')
# ax[0].set_ylim((0,2))
# # plt.ylim((2,8))
# ax[0].legend()
# ax[1].legend()
# plt.show()


## save results for the resnet


In [0]:
text = 'best_med_error: {}\ncorresponding_accuracy: {}\n'.format(best_med_error_resNet, corresponding_accuracy_resNet)
with open(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_val_best', "w") as text_file:
    text_file.write(text)



np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_med_error_val', med_errors_val_resNet)
np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_loss_val', losses_val_resNet)

# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_bin_losses', bin_losses)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_delta_losses', delta_losses)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_length_losses_x', length_losses_x)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_length_losses_y', length_losses_y)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_projection_losses', projection_losses)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_class_shape_losses', class_shape_losses)	
# np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_embedding_losses', embedding_losses)	
np.save(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_losses_train', losses_train_resNet)

# TEST on test data

## load the data

In [0]:
pointclouds, valid_im_ids_test, shape_idxs_test, true_rots_test, bboxes_test, images_test, num_test_images = load_data(CLASS, shapes_per_class, split='Test/')

In [0]:
class ImageDatasetTest(torch.utils.data.Dataset):
  def __init__(self):
    self.test_im_ids = valid_im_ids_test
    self.test_true_rots = true_rots_test
    self.test_shape_idxs = shape_idxs_test
    self.test_bboxes = bboxes_test
    self.images = images_test

    length_classes = []
    for cls in CLASS:
      length_classes.append(num_test_images[cls])

    self.test_class_idxs = np.repeat(range(len(CLASS)), length_classes)
    self.test_shape_idxs = self.test_shape_idxs.astype(int)
  
  def __len__(self):
    return len(self.test_im_ids)

  def __getitem__(self, index):
    sample = {}
    sample['im_id'] = self.test_im_ids[index]
    sample['rot'] = self.test_true_rots[index]
    sample['shape_index'] = self.test_shape_idxs[index]
    sample['class_index'] = self.test_class_idxs[index]
    sample['bbox'] = self.test_bboxes[index]

    sample['image_original'] = self.images[index]
    sample = transform_image_test(sample)

    del sample['image_original']

    sample['orientation'] = torch.FloatTensor(sample['rot'])

    sample = get_bin_activations(sample)
    sample = get_true_delta_rotations(sample)

    sample = get_embedding(sample)
    return sample

In [0]:
dataloader_test = torch.utils.data.DataLoader(ImageDatasetTest(), batch_size=BATCH_SIZE_RESNET)

## load Net

In [0]:
resNet = torchvision.models.wide_resnet50_2(pretrained=True)
resNet.fc = nn.Sequential(nn.Dropout(p=DROPOUT_RATE), nn.Linear(2048, EMBEDDING_SIZE))
resNet = resNet.to(device)


# # to still have dropout in evaluation:
# for m in resNet.modules():
#   if m.__class__.__name__.startswith('Dropout'):
#     m.train()

In [44]:
resNet.load_state_dict(torch.load(PATH_NETWORK + EXPERIMENT_NAME + '_resNet_best_med_error.pt'))

<All keys matched successfully>

In [45]:
resNet.eval()
torch.set_grad_enabled(False)

In [46]:
voxNet = VoxNet()	
voxNet = voxNet.to(device)

if LOAD_VOXNET and NUM_EPOCHS_VOXNET == 0:	
  voxNet.load_state_dict(torch.load(PATH_NETWORK + VOXNET_LOADED_FROM + '_voxNet_best_med_error.pt'))	
else:	
  voxNet.load_state_dict(torch.load(PATH_NETWORK + EXPERIMENT_NAME + '_voxNet_best_med_error.pt'))	

voxNet.eval()	
torch.set_grad_enabled(False)

## run on data


In [0]:
geodesic_distances = []
accurate_count = 0
correct_shapes = 0
correct_classes = 0

class_correct = []
shape_correct = []


accurate_count_individual = {'aeroplane':0, 'bicycle':0, 'boat':0, 'bottle':0, 'bus':0, 'car':0, 
                             'chair':0, 'diningtable':0, 'motorbike':0, 'sofa':0, 'train':0, 'tvmonitor':0}
geodesic_distances_individual = {'aeroplane':[], 'bicycle':[], 'boat':[], 'bottle':[], 'bus':[], 'car':[], 
                             'chair':[], 'diningtable':[], 'motorbike':[], 'sofa':[], 'train':[], 'tvmonitor':[]}
correct_classes_individual = {'aeroplane':0, 'bicycle':0, 'boat':0, 'bottle':0, 'bus':0, 'car':0, 
                             'chair':0, 'diningtable':0, 'motorbike':0, 'sofa':0, 'train':0, 'tvmonitor':0}
correct_shapes_individual = {'aeroplane':0, 'bicycle':0, 'boat':0, 'bottle':0, 'bus':0, 'car':0, 
                             'chair':0, 'diningtable':0, 'motorbike':0, 'sofa':0, 'train':0, 'tvmonitor':0}                             

for batch in iter(dataloader_test):

  logits = resNet(batch['image'])
  bin_result = voxNet.results_from_embedding(logits)

  gds, acc_count = geodesic_distances_batch_from_rots(batch, bin_result)

  geodesic_distances.extend(gds)
  accurate_count += acc_count


  predictions_classes_shapes = (torch.argmax(bin_result['class_shape'].to("cpu"), axis=1)).type(torch.IntTensor)
  predictions_classes = (predictions_classes_shapes / MAX_NUM_SHAPES).type(torch.IntTensor)
  correct_classes += torch.sum(torch.eq(predictions_classes, batch['class_index'].to("cpu")))
  class_correct.extend(torch.eq(predictions_classes, batch['class_index'].to("cpu")))

  predictions_shapes = (predictions_classes_shapes - predictions_classes * MAX_NUM_SHAPES).type(torch.IntTensor)
  correct_shapes += torch.sum(torch.eq(predictions_shapes, batch['shape_index'].to("cpu")))
  shape_correct.extend(torch.eq(predictions_shapes, batch['shape_index'].to("cpu")))

  for cls_idx, cls in enumerate(CLASS):
    geodesic_distances_individual[cls].extend(gds[torch.eq(batch['class_index'], cls_idx)])
    accurate_count_individual[cls] += torch.sum((gds[torch.eq(batch['class_index'], cls_idx)]) < 0.52)

    true_class = batch['class_index'].to("cpu").clone()
    true_class[torch.ne(true_class,cls_idx)] = -1
    correct_classes_individual[cls] += torch.sum(torch.eq(predictions_classes, true_class))

    true_shape = batch['shape_index'].to("cpu").clone()
    true_shape[torch.ne(true_class,cls_idx)] = -1
    correct_shapes_individual[cls] += torch.sum(torch.eq(predictions_shapes, true_shape))



# print(accurate_count_individual)
# print(geodesic_distances_individual)
# print(correct_classes_individual)
# print(correct_shapes_individual)


med_error_test = torch.median(torch.FloatTensor(geodesic_distances))
accuracy_test = float(accurate_count) / len(dataloader_test.dataset)
shape_accuracy_test = float(correct_shapes) / len(dataloader_test.dataset)
class_accuracy_test = float(correct_classes) / len(dataloader_test.dataset)

# print(med_error_test, accuracy_test, shape_accuracy_test)

In [48]:
print('med error: {}'.format(torch.median(torch.FloatTensor(geodesic_distances))))
print('accuracy: {}'.format(float(accurate_count) / len(dataloader_test.dataset)))

med error: 0.11389602720737457
accuracy: 0.8926820572243516


In [0]:
text = 'best_med_error: {}\ncorresponding_accuracy: {}\ncorresponding_correct_classes: {}\ncorresponding_correct_shapes: {}\n'.format(torch.median(torch.FloatTensor(geodesic_distances)), float(accurate_count) / len(dataloader_test.dataset), float(correct_classes) / len(dataloader_test.dataset), float(correct_shapes) / len(dataloader_test.dataset))
with open(PATH_RESULTS + EXPERIMENT_NAME + '_resNet_test', "w") as text_file:
    text_file.write(text)

# save data

In [0]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [0]:
if WITHOUT_EMBEDDING or NUM_EPOCHS_VOXNET == 0:	
  best_med_error_voxNet = torch.FloatTensor([np.nan])	
  corresponding_accuracy_voxNet = torch.FloatTensor([np.nan])	
  corresponding_correct_classes_voxNet = np.nan	
  corresponding_correct_shapes_voxNet = np.nan	

In [0]:
import pandas as pd

		
# check whether the results-file exists already	
# if so, append data, if not, make file	
result_file_is_new = True	
if NAME_RESULT_FILE + '.pkl' in os.listdir(PATH_RESULTS):
  print('result file ' + NAME_RESULT_FILE + ' is NOT new!')
  result_file_is_new = False
else:
  print('result file ' + NAME_RESULT_FILE + ' is new!')

if not result_file_is_new:	
  last_df = load_obj(PATH_RESULTS + NAME_RESULT_FILE)	
  backup_df = last_df.copy()


df = pd.DataFrame({'Experiment':[EXPERIMENT_NAME],
                   'CLASS': [CLASS],
                   'VOXNET_LOADED_FROM' : [VOXNET_LOADED_FROM],
                   'RESNET_LOADED_FROM' : [RESNET_LOADED_FROM],
                   'Test med error': [med_error_test.item()],
                   'Test accuracy': [accuracy_test],
                   'Test correct classes': [class_accuracy_test],
                   'Test correct shapes': [shape_accuracy_test],
                   'JITTER_FACTOR':[JITTER_FACTOR],
                   'DROPOUT_RATE':[DROPOUT_RATE],
                   'ALPHA': [ALPHA],
                   'BETA': [BETA],
                   'WEIGHT_DELTA_LOSS': [WEIGHT_DELTA_LOSS],
                   'WEIGHT_EMBEDDING_LOSS':[WEIGHT_EMBEDDING_LOSS],
                   'WEIGHT_SHAPE_LOSS': [WEIGHT_SHAPE_LOSS],
                   'WEIGHT_SHAPE_LOSS_VOXNET':[WEIGHT_SHAPE_LOSS_VOXNET],
                   'DISENTANGLED':[DISENTANGLED],
                   'EMBEDDING_SIZE': [EMBEDDING_SIZE],
                   'SHAPE_EMBEDDING_SIZE':[SHAPE_EMBEDDING_SIZE],
                   'BATCH_SIZE_VOXNET': [BATCH_SIZE_VOXNET],
                   'BATCH_SIZE_RESNET': [BATCH_SIZE_RESNET],
                   'NUM_EPOCHS_VOXNET': [NUM_EPOCHS_VOXNET],
                   'LR_VOXNET':[LR_VOXNET],
                   'EPOCH_CHANGE_LR_VOXNET':[EPOCH_CHANGE_LR_VOXNET],
                   'NUM_EPOCHS_RESNET': [NUM_EPOCHS_RESNET],
                   'LR_RESNET':[LR_RESNET],	
                   'EPOCH_CHANGE_LR_RESNET' : [EPOCH_CHANGE_LR_RESNET],
                   'Stage 1 med error': [best_med_error_voxNet.item()],
                   'Stage 1 accuracy': [corresponding_accuracy_voxNet.item()],
                   'Stage 1 correct classes': [corresponding_correct_classes_voxNet],
                   'Stage 1 correct shapes': [corresponding_correct_shapes_voxNet],
                   'Stage 2 val med error': [best_med_error_resNet.item()], 
                   'Stage 2 val accuracy': [corresponding_accuracy_resNet],
                   'Stage 2 correct classes': [corresponding_correct_classes_resNet],
                   'Stage 2 correct shapes': [corresponding_correct_shapes_resNet]
                   }, index=[0])

if not result_file_is_new:	
  last_df = last_df.append(df)	
else:	
  last_df = df

save_obj(last_df, PATH_RESULTS + NAME_RESULT_FILE)

In [0]:
# save individual scores

temp_name_individual_result_file = NAME_RESULT_FILE + '_individual'	

if not result_file_is_new:	
  last_df_individual = load_obj(PATH_RESULTS + temp_name_individual_result_file)	
  backup_df_individual = last_df_individual.copy()	


df_individual = pd.DataFrame({'Experiment':[EXPERIMENT_NAME]}, index=[0])

for cls in CLASS:
  df_individual = df_individual.assign(**{(cls+' med error'): float(torch.median(torch.FloatTensor(geodesic_distances_individual[cls]))), 
                                       cls+' accuracy':float(accurate_count_individual[cls]) / num_test_images[cls],
                                       cls+' correct class': float(correct_classes_individual[cls]) / num_test_images[cls],
                                       cls+' correct shape': float(correct_shapes_individual[cls]) / num_test_images[cls]})
 
if not result_file_is_new:	
  last_df_individual = last_df_individual.append(df_individual, ignore_index=True)	
else:	
  last_df_individual = df_individual

save_obj(last_df_individual, PATH_RESULTS + temp_name_individual_result_file)